# Prepare environment

In [1]:
# TODO: current dataset contains sequences of length 2: remove them

!pip install git+https://github.com/katarinagresova/ensembl_scraper.git

  Cloning https://github.com/katarinagresova/ensembl_scraper.git to /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-p2yyoi4a
  Running command git clone --filter=blob:none -q https://github.com/katarinagresova/ensembl_scraper.git /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-p2yyoi4a
  Resolved https://github.com/katarinagresova/ensembl_scraper.git to commit 8239db77fd3f0374300cd4d1efac319fb28f3535
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for scraper, since package 'wheel' is not installed.
  Attempting uninstall: scraper
    Found existing installation: scraper 0.0.1
    Uninstalling scraper-0.0.1:
      Successfully uninstalled scraper-0.0.1
    Running setup.py install for scraper ... done


In [2]:
!wget https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt

--2021-10-18 16:58:53--  https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt
Prevádza sa raw.githubusercontent.com (raw.githubusercontent.com) na IP adresu... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Pripájanie k raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... pripojené.
HTTP požiadavka odoslaná, čakám na odpoveď... 200 OK
Dĺžka:  337 [text/plain]
Ukladá sa do: `requirements.txt.1'

requirements.txt.1  100%[===================>]     337  --.-KB/s    za 0s      

2021-10-18 16:58:53 (11,9 MB/s) - `requirements.txt.1' uložené [337/337]



In [3]:
!pip install -r requirements.txt

# Create config file

In [4]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "homo_sapiens": {
            "external_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

# Prepare directories

In [5]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/demo_human_enhancers_ensembl/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

# Run tool

In [6]:
!python -m scraper.ensembl_scraper -c user_config.yaml

Processing feature files:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:download_file(): Going to download file from path ftp://ftp.ensembl.org/pub/release-100/mysql/regulation_mart_100/hsapiens_external_feature__external_feature__main.txt.gz
INFO:root:download_file(): File downloaded to path ../../datasets//tmp//homo_sapiens_external_feature.txt.gz.
INFO:root:parse_feature_file(): Going to parse file ../../datasets//tmp//homo_sapiens_external_feature.txt.gz
INFO:root:parse_feature_file(): Done parsing file ../../datasets//tmp//homo_sapiens_external_feature.txt.gz


Processing feature types:   0%|                           | 0/2 [00:00<?, ?it/s]INFO:root:find_sequences(): Going to find sequences based on genomic loci.
INFO:root:download_2bit_file(): Going to download 2bit file hg38
INFO:root:download_2bit_file(): File for hg38 downloaded to path ../../datasets//tmp/hg38.2bit.
INFO:root:find_sequences(): Done finding sequences.
INFO:root:remove_low_quality(): Going to 

# Cleaning

In [7]:
!mv ../../datasets/homo_sapiens_external_feature_enhancer ../../datasets/demo_human_enhancers_ensembl/

In [8]:
!rm requirements.txt user_config.yaml

In [9]:
!rm -rf ../../datasets/tmp/

In [16]:
!rm -rf ../../datasets/homo_sapiens_external_feature_transcription_start_site

# Final reformating

  * gzip all CSV files
  * add extra formatting to yaml config file

In [11]:
!find ../../datasets/demo_human_enhancers_ensembl/ -type f -name "*.csv" -exec gzip {} \;

In [12]:
with open("../../datasets/demo_human_enhancers_ensembl/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'}},
 'version': 0}

In [13]:
config['classes']['positive']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 
config['classes']['negative']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'}},
 'version': 0}

In [14]:
with open("../../datasets/demo_human_enhancers_ensembl/metadata.yaml", 'w') as handle:
  yaml.dump(config, handle)